In [21]:
import pandas as pd
import numpy as np

In [22]:
stud= pd.read_csv('data.csv', sep=';')
stud.head()
stud.shape
stud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4424 entries, 0 to 4423
Data columns (total 37 columns):
 #   Column                                          Non-Null Count  Dtype  
---  ------                                          --------------  -----  
 0   Marital status                                  4424 non-null   int64  
 1   Application mode                                4424 non-null   int64  
 2   Application order                               4424 non-null   int64  
 3   Course                                          4424 non-null   int64  
 4   Daytime/evening attendance	                     4424 non-null   int64  
 5   Previous qualification                          4424 non-null   int64  
 6   Previous qualification (grade)                  4424 non-null   float64
 7   Nacionality                                     4424 non-null   int64  
 8   Mother's qualification                          4424 non-null   int64  
 9   Father's qualification                   

In [23]:
#verificar a taxa de evasão escolar
stud['Target'].value_counts(1) # esse 1 obtem o percentual

Graduate    0.499322
Dropout     0.321203
Enrolled    0.179476
Name: Target, dtype: float64

<p> Aqui tiramos que a taxa de evasão, o dropout é de 32%. </p><p>Em seguida, foi pedido para que verificassemos se a taxa de inflação para cada tipo de target poderia indicar essa evasão.</p>

In [24]:
#Média de inflação para cada tipo de target
stud.groupby('Target')['Inflation rate'].mean()

Target
Dropout     1.283955
Enrolled    1.211713
Graduate    1.197918
Name: Inflation rate, dtype: float64

<p> A taxa foi baixa e não parece ser significativa, a sugestão então foi que talvez a situação econômica, o estado civil ou se o estudante mora ou não com os pais poderia ter alguma relação com isso. E para fazer essa análise mais ampla, precisaremos aprender pivot_table no Pandas!! </p>

In [25]:
stud.pivot_table(
    index='Target', 
    columns=['Marital status'], 
    values='Inflation rate', 
    aggfunc=np.mean
    )

Marital status,1,2,3,4,5,6
Target,,,,,,
Dropout,1.282601,1.300559,1.40,1.269048,1.354545,0.875
Enrolled,1.220694,1.003846,2.55,1.593750,-0.333333,1.400
Graduate,1.193499,1.216216,-0.80,1.163636,2.181818,-0.300



O método pivot_table no pandas é usado para criar uma tabela dinâmica (pivot table) a partir de um DataFrame, permitindo agregar e reorganizar os dados de forma flexível.

Como funciona a pivot_table
A função pivot_table tem alguns parâmetros principais:

index: Especifica uma ou mais colunas que serão usadas como índice na tabela dinâmica. Os valores dessas colunas formam as linhas da tabela.

columns: Especifica uma ou mais colunas cujos valores serão usados para criar as colunas da tabela dinâmica.

values: Especifica a(s) coluna(s) cujos valores serão agregados de acordo com a função de agregação (como mean, sum, etc.).

aggfunc: Define a função de agregação a ser usada (por exemplo, np.mean para calcular a média, np.sum para somar, etc.). O padrão é mean.

In [26]:
stud.pivot_table(
    index='Target',
    columns=['Daytime/evening attendance\t', 'Marital status'],
    values= 'Previous qualification (grade)',
    aggfunc=np.mean
)

Daytime/evening attendance	           0                                        \
Marital status                        1           2      3           4      5   
Target                                                                          
Dropout                      128.471774  130.476471    NaN  127.733333  120.0   
Enrolled                     128.863415  133.296000    NaN  135.400000    NaN   
Graduate                     132.313333  131.133784  120.0  138.388235  120.0   

Daytime/evening attendance	                  1                                 \
Marital status                   6           1           2      3           4   
Target                                                                          
Dropout                      122.5  131.537736  129.937838  135.0  130.753333   
Enrolled                     133.1  131.172607  129.177778  154.0  137.412500   
Graduate                     133.1  134.495393  130.505405    NaN  126.593750   

Daytime/evening attendance	                   
Marital status                      5      6  
Target                                        
Dropout                      143.5500  133.1  
Enrolled                     128.7000    NaN  
Graduate                     132.1375    NaN

Formatação condicional


In [27]:
#criando dataframe
df= pd.DataFrame({
    'A': [9, -7, 5],
    'B': [-1,3,-4]
})
df

,A,B
0,9,-1
1,-7,3
2,5,-4


In [28]:
#definindo função cor
def color_positive_negative(val):
    color='green' if val > 0 else 'red'
    return 'color: %s' % color

#aplicando a função de colocaração no Dataframe

style_df= df.style.applymap(color_positive_negative)
display(style_df)

,A,B
0,9,-1
1,-7,3
2,5,-4


Definindo função que destaca o maximo e o minimo para cada coluna


In [29]:
#definindo a função que destaca max e min

def highlight_max_min(s):
    is_max =s ==s.max()
    is_min =s ==s.min()
    return['color: green' if v_max else 'color: red' if v_min else ''for v_max, v_min in zip (is_max, is_min)]
#aplicando a função
style_df= df.style.apply(highlight_max_min)
display(style_df)

,A,B
0,9,-1
1,-7,3
2,5,-4


vamos criar uma nova pivolt table

In [30]:
pivot_df= stud.pivot_table(
    index='Marital status',
    columns= ['Target'],
    values='Previous qualification (grade)',
    aggfunc=np.mean
)

pivot_df

Target,Dropout,Enrolled,Graduate
Marital status,,,
1,131.216639,131.041111,134.381687
2,130.142458,131.157692,130.819595
3,135.000000,154.000000,120.000000
4,129.890476,136.406250,132.669697
5,141.409091,128.700000,128.827273
6,127.800000,133.100000,133.100000


Agora iremos utilizar uma função para marcar os maiores e menores valores de cada coluna, por isso iremos usar o apply invés do applymap

In [37]:
def highlight_max_min(data):
    styles= data.copy()
    for col in data.columns:
        max_val= data[col].max()
        min_val= data[col].min()
        styles[col]=['background-color: lightgreen' if v == max_val else 'background-color: yellow'
                     if v == min_val else '' for v in data[col]
                     ]
    return styles
#aplicando a função
styled_df= pivot_df.style.apply(highlight_max_min, axis=None)
display(styled_df)

Target,Dropout,Enrolled,Graduate
Marital status,,,
1,131.216639,131.041111,134.381687
2,130.142458,131.157692,130.819595
3,135.000000,154.000000,120.000000
4,129.890476,136.406250,132.669697
5,141.409091,128.700000,128.827273
6,127.800000,133.100000,133.100000


Usando funções aggs: função de agregação para colunas:

podemos usar multiplicas funções aggs desde que colocadas entre ['sum', 'mean', 'min']

In [38]:
stud.agg(['sum','mean','min'])

C:\Users\Thiag\AppData\Local\Temp\ipykernel_6696\1910189942.py:1: FutureWarning: ['Target'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  stud.agg(['sum','mean','min'])


,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
sum,5214.000000,82592.000000,7644.000000,3.918179e+07,3941.000000,20252.000000,586681.300000,8287.000000,86542.000000,98546.000000,...,2397.000000,27571.000000,35672.000000,19624.000000,45258.430117,665.000000,51168.600000,5432.800000,8.710000,DropoutGraduateDropoutGraduateGraduateGraduate...
mean,1.178571,18.669078,1.727848,8.856643e+03,0.890823,4.577758,132.613314,1.873192,19.561935,22.275316,...,0.541817,6.232143,8.063291,4.435805,10.230206,0.150316,11.566139,1.228029,0.001969,NaN
min,1.000000,1.000000,0.000000,3.300000e+01,0.000000,1.000000,95.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.600000,-0.800000,-4.060000,Dropout


É bem parecido com o describe, mas permite mais flexibilização, pois permite você trabalhar com funções próprias e escolher quais quer analisar.

In [39]:
#retorna a diferença entre o valor max e min da serie
def amplitude(series):
    return series.max() - series.min()

stud.agg(['sum', 'mean', 'min','max', amplitude])

C:\Users\Thiag\AppData\Local\Temp\ipykernel_6696\1293046201.py:4: FutureWarning: ['Target'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  stud.agg(['sum', 'mean', 'min','max', amplitude])


,Marital status,Application mode,Application order,Course,Daytime/evening attendance\t,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,Father's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
sum,5214.000000,82592.000000,7644.000000,3.918179e+07,3941.000000,20252.000000,586681.300000,8287.000000,86542.000000,98546.000000,...,2397.000000,27571.000000,35672.000000,19624.000000,45258.430117,665.000000,51168.600000,5432.800000,8.710000,DropoutGraduateDropoutGraduateGraduateGraduate...
mean,1.178571,18.669078,1.727848,8.856643e+03,0.890823,4.577758,132.613314,1.873192,19.561935,22.275316,...,0.541817,6.232143,8.063291,4.435805,10.230206,0.150316,11.566139,1.228029,0.001969,NaN
min,1.000000,1.000000,0.000000,3.300000e+01,0.000000,1.000000,95.000000,1.000000,1.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,7.600000,-0.800000,-4.060000,Dropout
max,6.000000,57.000000,9.000000,9.991000e+03,1.000000,43.000000,190.000000,109.000000,44.000000,44.000000,...,19.000000,23.000000,33.000000,20.000000,18.571429,12.000000,16.200000,3.700000,3.510000,Graduate
amplitude,5.000000,56.000000,9.000000,9.958000e+03,1.000000,42.000000,95.000000,108.000000,43.000000,43.000000,...,19.000000,23.000000,33.000000,20.000000,18.571429,12.000000,8.600000,4.500000,7.570000,NaN


Unindo diferentes Datasets
- dificilmente nas empresas todas as informações que precisamos estarão em um único dataset, por isso é imporante sabermos usar multimos datasetes e o SQL.

vamos emular isso criando data set de compras e geolocalização.


In [40]:
import pandas as pd

# DataFrame de Compras
compras = pd.DataFrame({
    'CompraID': [1, 2, 3, 4, 5],
    'ClienteID': [101, 102, 103, 101, 104],
    'Produto': ['Notebook', 'Smartphone', 'Tablet', 'Smartphone', 'Notebook'],
    'Valor': [2500.00, 1500.00, 1200.00, 800.00, 2700.00]
})

print(compras)


   CompraID  ClienteID     Produto   Valor
0         1        101    Notebook  2500.0
1         2        102  Smartphone  1500.0
2         3        103      Tablet  1200.0
3         4        101  Smartphone   800.0
4         5        104    Notebook  2700.0


In [41]:
# DataFrame de Geolocalização
geolocalizacao = pd.DataFrame({
    'ClienteID': [101, 102, 103, 104],
    'Cidade': ['São Paulo', 'Rio de Janeiro', 'Belo Horizonte', 'Curitiba'],
    'Latitude': [-23.55052, -22.9068, -19.9167, -25.4284],
    'Longitude': [-46.6333, -43.1729, -43.9345, -49.2733]
})

print(geolocalizacao)


   ClienteID          Cidade  Latitude  Longitude
0        101       São Paulo -23.55052   -46.6333
1        102  Rio de Janeiro -22.90680   -43.1729
2        103  Belo Horizonte -19.91670   -43.9345
3        104        Curitiba -25.42840   -49.2733


nesse caso groupby()[] o parenteses indica a coluna que você quer agrupar e o [] o valor que você ira agregar, o sum() mean() depois disso é a função que será aplicado dentro do parametro dado dentre []

In [43]:
#somando todas as compras por ID de clientes

compras_group= compras.groupby('ClienteID')['Valor'].sum().reset_index()
compras_group

,ClienteID,Valor
0,101,3300.0
1,102,1500.0
2,103,1200.0
3,104,2700.0


agora iremos fazer o merge para buscar informações de outros datasets.

In [44]:
compras_group_geo=compras_group.merge(geolocalizacao, on='ClienteID', how='left')
compras_group_geo

,ClienteID,Valor,Cidade,Latitude,Longitude
0,101,3300.0,São Paulo,-23.55052,-46.6333
1,102,1500.0,Rio de Janeiro,-22.90680,-43.1729
2,103,1200.0,Belo Horizonte,-19.91670,-43.9345
3,104,2700.0,Curitiba,-25.42840,-49.2733


as vezes também é necessário empilhar os dados, como nesse caso que teremos dois datasets com dados de compras de junho e julho e outro de agosto e semtembro, para esse caso podemos usar o concat, que irá jogar os dados das tabelas uma sobre a outra, ou uma do lado da outra


In [45]:
import pandas as pd

# DataFrame de Compras de Junho e Julho
compras_jun_jul = pd.DataFrame({
    'CompraID': [6, 7, 8],
    'ClienteID': [105, 106, 107],
    'Produto': ['Notebook', 'Smartphone', 'Tablet'],
    'Valor': [2600.00, 1300.00, 1500.00],
    'Data': ['2024-06-15', '2024-07-01', '2024-07-20']
})

print(compras_jun_jul)


   CompraID  ClienteID     Produto   Valor        Data
0         6        105    Notebook  2600.0  2024-06-15
1         7        106  Smartphone  1300.0  2024-07-01
2         8        107      Tablet  1500.0  2024-07-20


In [46]:
# DataFrame de Compras de Agosto e Setembro
compras_ago_set = pd.DataFrame({
    'CompraID': [9, 10, 11],
    'ClienteID': [108, 109, 110],
    'Produto': ['Notebook', 'Smartphone', 'Tablet'],
    'Valor': [2700.00, 1400.00, 1600.00],
    'Data': ['2024-08-05', '2024-08-20', '2024-09-10']
})

print(compras_ago_set)


   CompraID  ClienteID     Produto   Valor        Data
0         9        108    Notebook  2700.0  2024-08-05
1        10        109  Smartphone  1400.0  2024-08-20
2        11        110      Tablet  1600.0  2024-09-10


In [47]:
df_concat= pd.concat([compras_jun_jul, compras_ago_set])
df_concat

,CompraID,ClienteID,Produto,Valor,Data
0,6,105,Notebook,2600.0,2024-06-15
1,7,106,Smartphone,1300.0,2024-07-01
2,8,107,Tablet,1500.0,2024-07-20
0,9,108,Notebook,2700.0,2024-08-05
1,10,109,Smartphone,1400.0,2024-08-20
2,11,110,Tablet,1600.0,2024-09-10


Outras funções e funcionalidades do Jupyternotebook.

A função cut permite você criar categorias baseada em intervalos (segmentar e classficar [bins]).


In [52]:
#criando dataframe contendo valores

df= pd.DataFrame(
    {
        'Value': [1,2,3,4,5,6,7]
    }
)

#usando a função cut para criar intervalos

df['bin']= pd.cut(df['Value'], bins=[0,3,5,7], labels=['Low','Medium','High']) #adiciona uma nova coluna
df

,Value,bin
0,1,Low
1,2,Low
2,3,Low
3,4,Medium
4,5,Medium
5,6,High
6,7,High


Os intervalos são de 0-3 low, 3-5 medium, 5-7 high. Se você tivesse dados fora esses intervalos tipo um 8, daria NaN, valor ausente (missing).

Inves de passar os intervalos, você também pode dizer em quantas partes você gostaria que o campo continuo fosse quebrado.

In [54]:
#criando dataframe contendo valores

df= pd.DataFrame(
    {
        'Value': [1,2,3,4,5,6,7]
    }
)

#Quebrando os intervalos 
df['equals_bins']= pd.cut(df['Value'], bins=3)
df

,Value,equals_bins
0,1,"(0.994, 3.0]"
1,2,"(0.994, 3.0]"
2,3,"(0.994, 3.0]"
3,4,"(3.0, 5.0]"
4,5,"(3.0, 5.0]"
5,6,"(5.0, 7.0]"
6,7,"(5.0, 7.0]"


Mas também podemos criar rótulos e até dividir os bins em quantis etc..